In [1]:
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot.camera.zmq_camera import ZmqCamera as Camera

from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

In [2]:
import cv2 as cv
from center_tracker import PersonTracker
import torch
from jetbot import ObjectDetector


class Follower:

    def __init__(self):
        self.model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
        self.results = None
        self.trackable_objects = None
        self.tracked_to = None
        self.pt = None

    def init_tracking(self, bbox):
        centre_to = Follower._center(bbox)
        self.pt = PersonTracker(centre_to)

    def _predict(self, img):
        self.detections = self.model(img)

    def _post_process(self):
        boxes = []
        for detection in self.detections[0]:
            boxes.append(detection['bbox'])
            
        return boxes

    def track(self, img):

        self._predict(img)

        boxes = self._post_process()

        for box in boxes:
            Follower._draw_boxes(img, *box, color=(61, 254, 96))

        if self.pt is not None:
            self.trackable_objects = self.pt.update(boxes)
            self.tracked_to = self.trackable_objects.get(0, None)

            for to in self.trackable_objects.values():
                if to.ID == 0:
                    img = Follower._draw_id(img, to.ID, to.centroid, (18, 13, 212))
                elif to.disappeared_count > 0:
                    img = Follower._draw_id(img, to.ID, to.predicted_centroid, (253, 63, 28))
                else:
                    img = Follower._draw_id(img, to.ID, to.centroid, (61, 254, 96))

        return img

    @staticmethod
    def _draw_id(image, objectID, centroid, color):
        GREEN = color
        text = "ID {}".format(objectID)
        cv.putText(image, text, (int(centroid[0]) - 10, int(centroid[1]) - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, GREEN, 2)
        cv.circle(image, (int(centroid[0]), int(centroid[1])), 4, GREEN, -1)
        return image

    @staticmethod
    def _draw_boxes(image, x_min, y_min, x_max, y_max, color):
        cv.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        return image

    @staticmethod
    def _center(bbox):
        center_x = int((bbox[0] + bbox[2]) / 2.0)
        center_y = int((bbox[1] + bbox[3]) / 2.0)
        return (center_x, center_y)



In [3]:
def main():
    cap = Video

    while True:
        # reading the image from video capture
        img = camera.value

        img = follower.track(img)
        
        image_widget.value = bgr8_to_jpeg(img)
        display(image_widget)
        
        if follower.tracked_to is not None:
            if follower.tracked_to.box is not None:
                to_box = follower.tracked_to.box
                print("to box: ", to_box)

        if cv.waitKey(1) & 0xFF == ord('s'):
            cv.destroyAllWindows()
            tb_box = cv.selectROI("Select object for tracking", img, fromCenter=False, showCrosshair=False)
            yolo.init_tracking(tb_box)
            cv.destroyWindow("Select object for tracking")
            
        if 0xFF == ord('q'):
            break
            
        camera.unobserve_all()
    
    camera.stop()
    cv.destroyAllWindows()

In [4]:
follower = Follower()
camera = Camera.instance(width=300, height=300)
img_widget = widgets.Image(format='jpeg', width=300, height=300)
display(img_widget)

def execute(change):
    img = change['new']
    img = follower.track(img)
    img_widget.value = bgr8_to_jpeg(img)

execute({'new':camera.value})

Image(value=b'', format='jpeg', height='300', width='300')

In [5]:
camera.unobserve_all()
camera.observe(execute, names='value')